
## Import 

In [1]:
cd ..

/home/alex/Desktop/openclassroom/mentorat/__alexandre_gazagnes_mentorat/DS/P7/for-nora


In [2]:
import os, sys, pickle

from IPython.display import display

import requests

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt 
import seaborn as sns
sns.set()
import plotly.express as px

from sklearn.model_selection import * 
from sklearn.compose import *
from sklearn.pipeline import *
from sklearn.impute import * 
from sklearn.preprocessing import * 
from sklearn.ensemble import * 
from xgboost import XGBClassifier



# import shap

## Load data

In [3]:
url = "./data/cleaned/df.csv"

In [4]:
df = pd.read_csv (url)

In [5]:
df.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,22.0,1,0,7.2500,0,1,0,0,1
1,2,1,1,38.0,1,0,71.2833,1,0,1,0,0
2,3,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,4,1,1,35.0,1,0,53.1000,1,0,0,0,1
4,5,0,3,35.0,0,0,8.0500,0,1,0,0,1


## Prepare

In [6]:
df.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,22.0,1,0,7.2500,0,1,0,0,1
1,2,1,1,38.0,1,0,71.2833,1,0,1,0,0
2,3,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,4,1,1,35.0,1,0,53.1000,1,0,0,0,1
4,5,0,3,35.0,0,0,8.0500,0,1,0,0,1


In [7]:
X = df.drop(columns="Survived")
y = df.Survived

In [8]:
df_train, df_test  = train_test_split(df)

In [9]:
df_train.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
196,197,0,3,NaN,0,0,7.7500,0,1,0,1,0
23,24,1,1,28.0,0,0,35.5000,0,1,0,0,1
283,284,1,3,19.0,0,0,8.0500,0,1,0,0,1
748,749,0,1,19.0,1,0,53.1000,0,1,0,0,1
5,6,0,3,NaN,0,0,8.4583,0,1,0,1,0


In [10]:
df_train.shape

(668, 12)

In [11]:
df_test.shape

(223, 12)

In [12]:
X_train_PassengerId = df_train.PassengerId.values
y_train = df_train.Survived.values
X_train = df_train.drop(columns=["Survived","PassengerId" ], inplace=False)
X_train.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
196,3,NaN,0,0,7.7500,0,1,0,1,0
23,1,28.0,0,0,35.5000,0,1,0,0,1
283,3,19.0,0,0,8.0500,0,1,0,0,1
748,1,19.0,1,0,53.1000,0,1,0,0,1
5,3,NaN,0,0,8.4583,0,1,0,1,0


In [13]:
y_train[:10]

array([0, 1, 1, 0, 0, 0, 0, 1, 1, 0])

In [14]:
len(y_train)

668

In [15]:
len(X_train)

668

In [16]:
X_test_PassengerId = df_test.PassengerId.values
y_test = df_test.Survived.values
X_test = df_test.drop(columns=["Survived","PassengerId" ], inplace=False)
X_test.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
601,3,NaN,0,0,7.8958,0,1,0,0,1
397,2,46.0,0,0,26.0000,0,1,0,0,1
290,1,26.0,0,0,78.8500,1,0,0,0,1
280,3,65.0,0,0,7.7500,0,1,0,1,0
170,1,61.0,0,0,33.5000,0,1,0,0,1


## Model

In [17]:
imputer = KNNImputer().fit(X_train)

X_train = pd.DataFrame(imputer.transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(imputer.transform(X_test), columns=X_train.columns)
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 668 entries, 0 to 667
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      668 non-null    float64
 1   Age         668 non-null    float64
 2   SibSp       668 non-null    float64
 3   Parch       668 non-null    float64
 4   Fare        668 non-null    float64
 5   Sex_female  668 non-null    float64
 6   Sex_male    668 non-null    float64
 7   Embarked_C  668 non-null    float64
 8   Embarked_Q  668 non-null    float64
 9   Embarked_S  668 non-null    float64
dtypes: float64(10)
memory usage: 52.3 KB


In [18]:
grid = GridSearchCV(RandomForestClassifier(), {}, cv=10, return_train_score=True, n_jobs=-1)

## Train

In [19]:
grid.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={}, return_train_score=True)

In [20]:
res = pd.DataFrame(grid.cv_results_)
cols = [i for i in res.columns if ("split" not in i) ]
res = res.loc[:, cols]
res

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
0,0.243605,0.043235,0.015795,0.003505,{},0.799344,0.042631,1,0.984697,0.001635


## Test

In [21]:
grid.score(X_train, y_train)

0.9835329341317365

In [22]:
model = grid.best_estimator_
model

RandomForestClassifier()

In [23]:
model.fit(X_train, y_train)

RandomForestClassifier()

In [24]:
model.score(X_test, y_test)

0.8071748878923767

# export

In [25]:
with open("./models/model.pk", "wb") as f : 
    pk = pickle.dumps(model)
    f.write(pk)

In [26]:
display(len(X_train))
display(len(X_train_PassengerId))
display(len(y_train))

668

668

668

In [27]:
len(y_train)

668

In [28]:
X_train["Survived"] = y_train
X_train["PassengerId"] = X_train_PassengerId
X_train.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Survived,PassengerId
0,3.0,35.9,0.0,0.0,7.7500,0.0,1.0,0.0,1.0,0.0,0,197
1,1.0,28.0,0.0,0.0,35.5000,0.0,1.0,0.0,0.0,1.0,1,24
2,3.0,19.0,0.0,0.0,8.0500,0.0,1.0,0.0,0.0,1.0,1,284
3,1.0,19.0,1.0,0.0,53.1000,0.0,1.0,0.0,0.0,1.0,0,749
4,3.0,35.9,0.0,0.0,8.4583,0.0,1.0,0.0,1.0,0.0,0,6


In [29]:
X_train.isna().sum()

Pclass         0
Age            0
SibSp          0
Parch          0
Fare           0
Sex_female     0
Sex_male       0
Embarked_C     0
Embarked_Q     0
Embarked_S     0
Survived       0
PassengerId    0
dtype: int64

In [30]:
X_train.to_csv("./data/cleaned/df_train.csv", index=False)

In [31]:
X_test["Survived"] = y_test
X_test["PassengerId"] = X_test_PassengerId
X_test.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Survived,PassengerId
0,3.0,29.0,0.0,0.0,7.8958,0.0,1.0,0.0,0.0,1.0,0,602
1,2.0,46.0,0.0,0.0,26.0000,0.0,1.0,0.0,0.0,1.0,0,398
2,1.0,26.0,0.0,0.0,78.8500,1.0,0.0,0.0,0.0,1.0,1,291
3,3.0,65.0,0.0,0.0,7.7500,0.0,1.0,0.0,1.0,0.0,0,281
4,1.0,61.0,0.0,0.0,33.5000,0.0,1.0,0.0,0.0,1.0,0,171


In [32]:
X_test.isna().sum()

Pclass         0
Age            0
SibSp          0
Parch          0
Fare           0
Sex_female     0
Sex_male       0
Embarked_C     0
Embarked_Q     0
Embarked_S     0
Survived       0
PassengerId    0
dtype: int64

In [33]:
X_test.to_csv("./data/cleaned/df_test.csv", index=False)